In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go

# Load SpaceX launch data (replace with your data path)
df = pd.read_csv("SpaceX_launch_data.csv")

# Available launch sites (extract from unique values in 'Launch_Site' column)
launch_sites = df['Launch_Site'].unique()

# Layout of the dashboard
app = dash.Dash(__name__)
app.layout = html.Div([
    # Title
    html.H1("SpaceX Launch Data Analysis", style={'text-align': 'center'}),
    # Launch site dropdown
    dcc.Dropdown(
        id='launch-site-dropdown',
        options=[{'label': site, 'value': site} for site in launch_sites],
        value='ALL'  # Default value to display all data initially
    ),
    # Success rate pie chart
    dcc.Graph(id='success-pie-chart'),
    # Payload range slider
    dcc.RangeSlider(
        id='payload-slider',
        min=df['PayloadMassKg'].min(),
        max=df['PayloadMassKg'].max(),
        value=[df['PayloadMassKg'].min(), df['PayloadMassKg'].max()],  # Default range
        marks={str(mass): f'{mass} kg' for mass in range(0, df['PayloadMassKg'].max()+1000, 5000)}  # Display marks every 5000 kg
    ),
    # Success vs Payload scatter plot
    dcc.Graph(id='success-payload-scatter-chart'),
])

# Callback function for success rate pie chart based on selected launch site
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('launch-site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        filtered_df = df.copy()  # Copy entire data for 'ALL'
    else:
        filtered_df = df[df['Launch_Site'] == selected_site]  # Filter by selected site

    # Calculate success rate
    successful_launches = filtered_df[filtered_df['Landing_Outcome'] == 'Success']
    total_launches = len(filtered_df)
    success_rate = (successful_launches.shape[0] / total_launches) * 100

    # Create pie chart
    fig = go.Figure(
        data=[go.Pie(labels=['Success', 'Failure'], values=[success_rate, 100 - success_rate])]
    )
    fig.update_layout(title='Launch Success Rate')
    return fig

# Callback function for scatter plot based on payload range
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    Input('payload-slider', 'value')
)
def update_scatter_chart(payload_range):
    min_payload, max_payload = payload_range
    filtered_df = df[(df['PayloadMassKg'] >= min_payload) & (df['PayloadMassKg'] <= max_payload)]

    # Create scatter plot with coloring based on landing outcome
    fig = px.scatter(
        filtered_df, x='PayloadMassKg', y='Flight_Number', color='Landing_Outcome',
        title='Payload Mass vs Flight Number'
    )
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
